In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#libs
import matplotlib.pyplot as plt
from matplotlib import colormaps
import numpy as np
import os
import torch
import torchvision
from torchmetrics.classification import JaccardIndex


#our classes
import utils #contains sam_utils, visual_utils, and other utility functions
from datasets.dataset_loading import CocoLoader,get_coco_split
import segmentation_models

#sam
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator


In [ ]:
# CUDA tests
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # might not be viable, check again!

print("CUDA available:" + str(torch.cuda.is_available()))
from torch.utils.cpp_extension import CUDA_HOME

print("CUDA_HOME:" + str(CUDA_HOME))
current_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
print(available_gpus)
print(torch.cuda.device_count())

In [ ]:
transforms = None
coco_train_dataset = CocoLoader(get_coco_split(split = 'val'), transform=transforms)


### SAM wrapper

In [ ]:
sam_wrapper = segmentation_models.SamWrapper(device="cuda", model="b")

In [ ]:
to_show = 2
offset = 1  # from the beginning of the dataset

# loads the wanted images
items = coco_train_dataset.get_amount(amount=to_show, offset=offset)

results_all = sam_wrapper.infer_masks(items, boxes=None)  # if no promptr, then gt box

for i, item in enumerate(items):  # visualise single image and masks on a grid
    # inferred results
    results = results_all[i]
    titles = ["Score " + str(sc) for sc in results["scores"]]

    # image and GT
    img = item["image"]
    gt_boxes = item["annotations"]["boxes"]

    utils.grid_masks_boxes(
        image=img, masks=results["masks"], titles=titles, boxes=gt_boxes, scale=4
    )

In [ ]:
to_show = 60  # how many images to show
offset = 0  # from the beginning of the dataset

show=False
IoU_threshold = 0.3

# initialise metrics class(es)
IoU_metric = JaccardIndex("binary")
items = coco_train_dataset.get_amount(amount=to_show, offset=offset)

results = sam_wrapper.infer_masks(items)

# compare resulting masks with the GT masks, calculate IoU. save those with low IoU
bad_masks = []
cats = []
IoU_all=[]
for result, item in zip(results, items): 
    IoUs_my = utils.get_IoU_multiple(result["masks"], item["annotations"]["masks"])
    IoU_all.extend(IoUs_my)

    for j, (inferred_mask, gt_mask) in enumerate(
        zip(result["masks"], item["annotations"]["masks"])
    ):
        IoU_tmp = IoU_metric.forward(inferred_mask, gt_mask)

        if IoU_tmp <= IoU_threshold:  # metrics for individual masks
            bad_masks.append(
                {
                    "image": item["image"],
                    "box": item["annotations"]["boxes"][j],
                    "inferred_mask": inferred_mask,
                    "gt_mask": gt_mask,
                }
            )
            cats.append(item["annotations"]["categories"][j])
print("Mean IoU:", IoU_metric.compute())
print("My IoU: ", np.mean(IoU_all))

# visualise the bad masks
# takes dict with image, box, inferred_mask, gt_mask
if show:
    cats = coco_train_dataset.translate_catIDs(cats)
    for cat, dict_bad_mask in zip(cats, bad_masks):
        utils.show_differences(dict_bad_mask, gt_class=cat, segmentation_model="SAM-1 (b)")

In [ ]:
pos = 7  # which image to show

item = coco_train_dataset[pos]
detected_boxes = item["annotations"]["boxes"]  # or detect here

# for every box, generate an array points and array of labels
middle_points = np.array([[utils.get_middle_point(box)] for box in detected_boxes])
point_labels = np.ones((len(detected_boxes), 1), dtype=np.int32)

# points: [ [[x,y],[x,y]] ,
#          [[x,y],[x,y]]...] , then inside [] for multiple images
# point_labels: [ [1],[1] ] , then inside [] for multiple images


# infer masks without and with point prompts.
results_box = sam_wrapper.infer_masks([item], boxes=[detected_boxes])
results_box_middle = sam_wrapper.infer_masks(
    [item], boxes=[detected_boxes], points=[middle_points], point_labels=[point_labels]
)


results_box = results_box[0]
results_box_middle = results_box_middle[0]
# visuals
for i, (point, box) in enumerate(zip(middle_points, detected_boxes)):
    # every box multiple times with different masks (+ point in the last)
    utils.grid_masks_boxes(
        image=item["image"],
        masks=[results_box["masks"][i], results_box_middle["masks"][i]],
        boxes=[box, box],  # same box
        titles=["Box", "Box + Point"],
        points=[None, point],  # point prompt only in the second one, None otherwise
        point_labels=[None, 1],
    )

#### SAM automatic mask generation

In [1]:
conf_dict = {"b":
 { "path": "/mnt/vrg2/imdec/models/sam1/sam_vit_b_01ec64.pth",
  "model_type": "vit_b"},
"h":{
  "path": "/mnt/vrg2/imdec/models/sam1/sam_vit_h_4b8939.pth",
  "model_type": "vit_h"}
}
_,sam_model = segmentation_models.prepare_sam(model='b',device="cuda",cfg = conf_dict) #load once
# then reinitialise just mask generator
# can be used in different models (sam and clip detection?)

NameError: name 'segmentation_models' is not defined

In [ ]:
to_show = 4
offset = 7  # from the beginning of the dataset

# get images
items = coco_train_dataset.get_amount(amount=to_show, offset=offset)


# prepare automatic generator
mask_generator = segmentation_models.AutomaticSam(
    model=sam_model,
    points_per_side=32,
    pred_iou_thresh=0.9,  # rejects "bad" masks
    stability_score_thresh=0.95,  # rejects "bad" masks
    crop_n_layers=1,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=200,  # Requires open-cv to run post-processing
)

# segment all automatically using the generator above
automatic_segmentations = mask_generator.automatic_multiple_images(
    items
)

# visualise
for i, item in enumerate(items):
    utils.print_masks_boxes(
        image=item["image"], masks=automatic_segmentations[i]["masks"], boxes=None
    )